### 示例11-1

In [ ]:
import requests
import json
import pandas as pd
AK= "申请的密钥"
service = "http://api.map.baidu.com/geoconv/v1/?"
f = 1; t = 5
coords = [[120.000,30.000],
        [100.000,40.000],
        [115.000,35.000],
        [-115.000,35.000],
        [0.000,40.000]]
bd_lng_list = []
bd_lat_list = []
for coord in coords:
    x = coord[0]
    y = coord[1]
    parameters = f"coords={x},{y}&from={f}&to={t}&ak={AK}"
    url = service + parameters
    response = requests.get(url)
    s=response.text
    dic=json.loads(s)
    bd_lng = dic["result"][0]["x"]
    bd_lat = dic["result"][0]["y"]
    bd_lng_list.append(bd_lng)
    bd_lat_list.append(bd_lat)
    
frame = pd.DataFrame(coords,columns=["WGS_lng","WGS_lat"])
frame["bd_lng"] = bd_lng_list
frame["bd_lat"] = bd_lat_list
frame

### 示例11-2

In [ ]:
import pandas as pd
import requests
import json
service ="http://api.map.baidu.com/geocoding/v3/?"
output = "json"
AK= "申请的密钥"
def transform(x,y,f,t):
    service = "http://api.map.baidu.com/geoconv/v1/?"
    parameters = f"coords={x},{y}&from={f}&to={t}&ak={AK}"
    url = service + parameters
    response = requests.get(url)
    s=response.text
    dic=json.loads(s)
    x = dic["result"][0]["x"]
    y = dic["result"][0]["y"]
    return (x,y)
frame = pd.read_csv(r"c:\data\shanghai\POIs.csv")
bd_lng_list = []
bd_lat_list = []
for i in range(len(frame)):
    WGS_lng = frame.iloc[i]["WGS_lng"]
    WGS_lat = frame.iloc[i]["WGS_lat"]
    bd_lng,bd_lat = transform(WGS_lng,WGS_lat,1,5)
    bd_lng_list.append(bd_lng)
    bd_lat_list.append(bd_lat)
frame["bd_lng"] = bd_lng_list
frame["bd_lat"] = bd_lat_list
frame["d_lng"] = frame["WGS_lng"] - frame["bd_lng"]
frame["d_lat"] = frame["WGS_lat"] - frame["bd_lat"]
frame

### 示例11-3

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
import requests
service ="http://api.map.baidu.com/staticimage/v2?"
AK= "申请的密钥" 
lng,lat = 121.455871,31.222390
zoom = 15
parameters =f"ak={AK}&zoom={zoom}&center={lng},{lat}&markers={lng},{lat}\
&markerStyles=l,1,0xFF0000&coordtype=wgs84ll"
url = service + parameters
response = requests.get(url)
with open(r"c:\data\tmp\tmp1.jpg","wb") as f:
    f.write(response.content)
img = plt.imread(r"c:\data\tmp\tmp1.jpg")
plt.imshow(img)

### 示例11-4

In [ ]:
import sys
import requests
import json
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5 import uic
ui_file = r"c:\data\PyQt\geocode.ui" 
Ui_MainWindow, QtBaseClass = uic.loadUiType(ui_file)
class MainWindow(QMainWindow,Ui_MainWindow):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        self.comboBox.addItem("华东师范大学")
        self.comboBox.addItem("闵行区华东师范大学")
        self.comboBox.addItem("东川路500号")
        self.comboBox.addItem("闵行区东川路500号")
        self.comboBox.addItem("闵行区川路500号")
        self.comboBox.addItem("闵行区")

    def geocode(self):
        service ="http://api.map.baidu.com/geocoding/v3/?"
        output = "json"
        AK= "申请的密钥"
        address = self.comboBox.currentText()
        parameters = f"address={address}&output={output}&ak={AK}"
        url = service + parameters
        response = requests.get(url)
        text=response.text
        dic=json.loads(text)
        status = dic["status"]
        if status==0:
            lng = dic["result"]["location"]["lng"]
            lat = dic["result"]["location"]["lat"]
            self.lineEdit.setText(f"{lng:.6f},{lat:.6f}")
            zoom = 17
            parameters =f"ak={AK}&zoom={zoom}&center={lng},{lat}\
            &markers={lng},{lat}&markerStyles=l,1"
            service ="http://api.map.baidu.com/staticimage/v2?"
            url = service + parameters
            response = requests.get(url)
            img = QImage()
            img.loadFromData(response.content,"png")
            fit_img = img.scaled(self.imageLabel.size(),1)
            self.imageLabel.setPixmap(QPixmap(fit_img))
        else:
            self.lineEdit.setText("地理编码不成功")
            self.imageLabel.clear()
                
if not QApplication.instance():
    app = QApplication(sys.argv)
else:
    app = QApplication.instance()
window = MainWindow()
window.show()
app.exec()

### 示例11-5

In [ ]:
import requests
import json
import folium
from folium import CircleMarker
#坐标转换（WGS地理坐标转百度地理坐标）
def transform(x,y,f,t):
    service = "http://api.map.baidu.com/geoconv/v1/?"
    parameters = f"coords={x},{y}&from={f}&to={t}&ak={AK}"
    url = service + parameters
    response = requests.get(url)
    s=response.text
    dic=json.loads(s)
    x = dic["result"][0]["x"]
    y = dic["result"][0]["y"]
    return (x,y) 

#正向地理编码
service ="http://api.map.baidu.com/geocoding/v3/?"
output = "json"
AK= "申请的密钥"
address = "东方明珠电视塔"
parameters = f"address={address}&output={output}&ak={AK}"
url = service + parameters
response = requests.get(url)
text=response.text
dic=json.loads(text)
status = dic["status"]
if status==0:
    bd_lng = dic["result"]["location"]["lng"]
    bd_lat = dic["result"]["location"]["lat"]
else:
    print(f"{address}:地理编码不成功")

#对地理编码得到的坐标进行转换，并计算偏差
d_lng = bd_lng - transform(bd_lng,bd_lat,1,5)[0]
d_lat = bd_lat - transform(bd_lng,bd_lat,1,5)[1]
#根据偏差对坐标进行纠偏
WGS_lng = bd_lng + d_lng
WGS_lat = bd_lat + d_lat

#纠偏前和纠偏后的坐标作为CircleMarker点添加到地图控件
m = folium.Map(location=[WGS_lat,WGS_lng],
               tiles = "Stamenterrain",
               zoom_start=15)
CircleMarker(location=(bd_lat,bd_lng),
             radius=10,fill=True,fillOpacity=1).add_to(m)
CircleMarker(location=(WGS_lat,WGS_lng),
             radius=10).add_to(m)
m

### 示例11-6

In [ ]:
import requests
import json
service ="http://api.map.baidu.com/geocoder/v2/?"
output = "json"
pois = 1
AK= "申请的密钥"
lng,lat = 121.496076,31.244585
parameters = f"location={lat},{lng}&output={output}&pois={pois}&ak={AK}"
url = service + parameters
response = requests.get(url)
text = response.text
dic = json.loads(text)
status = dic["status"]
if status == 0:
    sematic_description = dic["result"]["sematic_description"]
    print("地址语义描述:",sematic_description)
    poi_list = dic["result"]["pois"]
    print("附近POI：")
    for poi in poi_list:
        print(poi["name"])
else:
    print("逆向地理编码不成功")

### 示例11-7

In [ ]:
import requests
import json
service ="http://api.map.baidu.com/geocoding/v3/?"
output = "json"
AK= "申请的密钥"
address = "闵行区华东师范大学"
parameters = f"address={address}&output={output}&ak={AK}"
url = service + parameters
response = requests.get(url)
text=response.text
dic=json.loads(text)
status = dic["status"]
if status==0:
    lng = dic["result"]["location"]["lng"]
    lat = dic["result"]["location"]["lat"]
else:
    print(f"{address}:地理编码不成功")
service ="http://api.map.baidu.com/place/v2/search?"
query = "酒店"
radius = 2000
page_size = 20
parameters = f"query={query}&location={lat},{lng}&radius={radius}\
&page_size={page_size}&output={output}&ak={AK}"
url = service + parameters
response = requests.get(url)
text=response.text
dic=json.loads(text)
if dic["status"]==0:
    print(f"共检索到{dic['total']}个POI：")
    for result in dic["results"]:
        print(result["name"],result["uid"])
else:
    print("POI检索不成功")

### 示例11-8

In [ ]:
import requests
import json
import pprint
service ="http://api.map.baidu.com/place/v2/detail?"
uid = "89fd4ea31856665ee5e96510"
scope = "2"
output = "json"
AK= "申请的密钥" 
parameters = f"uid={uid}&scope={scope}&output={output}&ak={AK}"
url = service + parameters
response = requests.get(url)
text=response.text
dic=json.loads(text)
if dic["status"]==0:
    pprint.pprint(dic["result"])
else:
    print("详情查看不成功")

### 示例11-9

In [ ]:
import requests
import json
service ="http://api.map.baidu.com/place/v2/search?"
query = "酒店"
page_size = 20
coord_type = 1
output = "json"
AK= "申请的密钥" 
lng = 121.41
total = 0
for i in range(6):
    lng = lng + 0.01
    lat = 30.99
    for j in range(10):
        lat = lat + 0.01
        bounds= f"{lat},{lng},{lat+0.01},{lng+0.01}"
        parameters = f"query={query}&page_size={page_size}\
        &bounds={bounds}&coord_type={coord_type}&output={output}&ak={AK}"
        url = service + parameters
        response = requests.get(url)
        text=response.text
        dic=json.loads(text)
        if dic["status"]==0:
            total = total + dic["total"]
            for result in dic["results"]:
                print(result["name"])
        else:
            print("POI检索不成功")
print(f"共检索到{total}个POI")

### 示例11-10

In [ ]:
import pandas as pd
import requests
import json
from shapely.geometry import LineString
import pprint
driving_service = "http://api.map.baidu.com/directionlite/v1/driving?"
output = "json"
AK= "申请的密钥" 
o_lat,o_lng = 31.036778,121.460011
d_lat,d_lng = 31.188081,121.442019
parameters = f"origin={o_lat},{o_lng}&destination={d_lat},{d_lng}&ak={AK}"
url = driving_service + parameters
response = requests.get(url)
text=response.text
dic=json.loads(text)
pprint.pprint(dic)
steps = dic["result"]["routes"][0]["steps"]
points = []
for step in steps:
    for point in step["path"].split(";"):
        x = float(point.split(",")[0])
        y = float(point.split(",")[1])
        points.append([x,y])
lineString = LineString(points)
display(lineString)

### 示例11-11

In [ ]:
import pandas as pd
import requests
import json
driving_service = "http://api.map.baidu.com/directionlite/v1/driving?"
output = "json"
AK= "申请的密钥" 
def cal_distance_duration(o_lat,o_lng,d_lat,d_lng):
    parameters = f"origin={o_lat},{o_lng}&destination={d_lat},{d_lng}&ak={AK}"
    url = driving_service + parameters
    response = requests.get(url)
    text=response.text
    dic=json.loads(text)
    distance = dic["result"]["routes"][0]["distance"]
    duration = dic["result"]["routes"][0]["duration"]
    return distance,duration
geocoding_service ="http://api.map.baidu.com/geocoding/v3/?"
address = "上海市政府"
parameters = f"address={address}&output={output}&ak={AK}"
url = geocoding_service + parameters
response = requests.get(url)
text=response.text
dic=json.loads(text)
status = dic["status"]
if status==0:
    d_lng = dic["result"]["location"]["lng"]
    d_lat = dic["result"]["location"]["lat"]
else:
    print(f"{address}:地理编码不成功")
frame = pd.read_csv(r"c:\data\shanghai\POIs.csv")
distances = []
durations = []
for address in frame["Address"]:
    parameters = f"address={address}&output={output}&ak={AK}"
    url = geocoding_service + parameters
    response = requests.get(url)
    text=response.text
    dic=json.loads(text)
    status = dic["status"]
    if status==0:
        o_lng = dic["result"]["location"]["lng"]
        o_lat = dic["result"]["location"]["lat"]
    else:
        print(f"{address}:地理编码不成功")
    distance = cal_distance_duration(o_lat,o_lng,d_lat,d_lng)[0]
    duration = cal_distance_duration(o_lat,o_lng,d_lat,d_lng)[1]
    distances.append(distance)
    durations.append(duration) 
frame["distance"]=distances
frame["duration"]=durations
frame